In [1]:
import openmm as mm
import shlex
import subprocess
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import pdbfixer
import openmm.app as app
import simtk.unit as unit
from openmmforcefields.generators import GAFFTemplateGenerator as gen
from openff.toolkit import Topology
import os

from openmm.app import CharmmPsfFile, CharmmCrdFile, CharmmParameterSet

In [2]:
base = '/scratch/htc/fsafarov/forcefield_charmm/charmm-gui/charmmgui-8ef5'

psf = CharmmPsfFile('/scratch/htc/fsafarov/forcefield_charmm/charmm-gui/charmmgui-8ef5/step5_assembly.psf')
crd = CharmmCrdFile('/scratch/htc/fsafarov/forcefield_charmm/charmm-gui/charmmgui-8ef5/step5_assembly.crd')



params = CharmmParameterSet(
    os.path.join(base, 'toppar/top_all36_prot.rtf'),
    os.path.join(base, 'toppar/par_all36m_prot.prm'),
    os.path.join(base, 'toppar/top_all36_lipid.rtf'),
    os.path.join(base, 'toppar/par_all36_lipid.prm'),
    os.path.join(base, 'toppar/toppar_water_ions.str'),
    os.path.join(base, '7v7/7v7.rtf'),
    os.path.join(base, '7v7/7v7.prm')
)

/scratch/htc/fsafarov/openmm_ff/lib/python3.12/site-packages/openmm/app/charmmparameterset.py:391: UserWarning: Replacing dihedral <DihedralType: k=0.27; phase=180.0; per=2> with <DihedralType: k=0.03; phase=180.0; per=2>
  warnings.warn('Replacing dihedral %r with %r' %


In [3]:
# pdb = app.PDBFile('/scratch/htc/fsafarov/forcefield_charmm/charmm-gui/charmmgui-8ef5/openmm/step5_input.pdb')
# psf.setBox(*pdb.topology.getPeriodicBoxVectors())
psf.setBox(82.6*unit.angstroms, 82.6*unit.angstroms, 100.0*unit.angstroms)

In [4]:
print(params.atom_types_int);

{-1: <openmm.app.internal.charmm.topologyobjects.AtomType object at 0x7fca31cd3020>}


In [5]:
system = psf.createSystem(params, nonbondedMethod=app.PME, nonbondedCutoff=1.0 * unit.nanometer, constraints = app.HBonds)

In [6]:
integrator_eq = mm.LangevinIntegrator(300.0*unit.kelvin, 1/unit.picoseconds, 2.0*unit.femtoseconds)

In [7]:
system.addForce(mm.MonteCarloBarostat(1*unit.atmosphere, 300*unit.kelvin))

9

In [8]:
platform = mm.Platform.getPlatformByName('CUDA')

In [9]:
simulation_eq = app.Simulation(psf.topology, system, integrator_eq)
simulation_eq.context.setPositions(crd.positions)

In [10]:
# Before minimization
state = simulation_eq.context.getState(getEnergy=True)
print("Potential energy before minimization:", state.getPotentialEnergy())

# Run minimization
simulation_eq.minimizeEnergy()

# After minimization
state = simulation_eq.context.getState(getEnergy=True)
print("Potential energy after minimization:", state.getPotentialEnergy())

Potential energy before minimization: 6.659178027074589e+20 kJ/mol
Potential energy after minimization: -832847.7754094787 kJ/mol


In [11]:
for step in range(50):  # Assuming 50 steps, each representing 1 ns of equilibration
    restraint_force = 5.0 * (50 - step) / 50.0  # Taper from 5 kcal/mol/Å² to 0
    # Apply harmonic restraints to protein heavy atoms
    # Restraint application code here (custom based on specific atoms or atom groups)

    simulation_eq.step(1000)  # Run for 1 ns (modify based on actual timestep size)


In [13]:
steps_per_microsecond = int(1e6 / (2.4 * 1e-3))  # Number of steps per µs with 2.4 fs timestep
simulation_eq.reporters.append(app.StateDataReporter('/scratch/htc/fsafarov/traj/output_1.log', 100, step=True, potentialEnergy=True, temperature=True))
simulation_eq.reporters.append(app.DCDReporter('/scratch/htc/fsafarov/traj/trajectory_1.dcd', 100))
simulation_eq.step(10000)  # Run for 100 µs

In [ ]:
with open('/scratch/htc/fsafarov/traj/trajectory_1.dcd', 'r') as file:
    content = file.read()

with open('output2.log', 'w') as file_:
    file_.write(content)

In [ ]:
with open('/scratch/htc/fsafarov/traj/trajectory_1.dcd', 'rb') as file:
    content = file.read()


with open('trajectory_2.dcd', 'wb') as file_:
    file_.write(content)